# Numerical exercices - M2 ICI 2021 (Guillaume Stirnemann)

Here, we will illustrate some of the class' concepts about stochastic dynamics in biomolecular systems. The present notebook can be executed using Jupyter. You don't need to know Python beforehands, and you should be able to quickly grasp some simple Python vocabulary. 

We first need to load some libraries. These are suites of functions that were already implemented and widely distributed so that everyone can use them. Here, this includes some very convenient and common libraries for numerical calculations (numpy) or plotting libraries (matplotlib). 

To execute a cell, press Shift + Enter at the same time. 

In [ ]:
import multiprocessing
import tqdm

import numpy as np
import scipy.stats as st
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from scipy.stats import poisson
import random

## 1. Depolymerization of actin  

Actin is a key constituent of the cytoskeleton, which allows a cell to perform many of its functions, to maintain its shape, and to move. In vivo, actin monomers polymerize to form long filaments that are about 16 $\mu$m long on average. It possesses a directional character: at the (-) end, it binds actin monomers linked to ADP; at the (+) end, it binds actin monomers linked to ATP. Each monomer is about 4-nm long. 

Below, you are provided with a code that can perform stochastic simulations of coin flipping. Given a probability $p$ to have heads, it returns the number of heads obtained for a given number number of trials $n$, and repeated $size$ times. 

In [ ]:
def simulate_coinflips(n, p, size=1):
    """
    Simulate n_samples sets of n coin flips with prob. p of heads.
    """
    n_heads = np.empty(size, dtype=int)
    for i in range(size):
        n_heads[i] = np.sum(np.random.random(size=n) < p)
    return n_heads


Now this is an example of a real coin-tossing experiments. Let's make 25 trials and repeat the experiment 10 times. $theor\_dist$ returns the expectations of $P(k,n)$, i.e., the probability to obtain $k$ heads among $n$ trials. It is basically the exact version of the Poisson formula we have seen in the class. 


In [ ]:
n = 
p = 
h_plot = np.arange(n+1)
theor_dist = st.binom.pmf(h_plot, n, p)


In [ ]:
size=
mu=n*p
h = simulate_coinflips(n, p, size)
plt.plot(h_plot, theor_dist,color='blue',marker='o',label='expected',linestyle='')
plt.plot(np.arange(h.max()+1), np.bincount(h)/size, color='orange',marker='o',label='P(h)',linestyle='')
plt.plot(h_plot, poisson.pmf(h_plot,mu),color='red',marker='o',label='expected',linestyle='')

plt.xlabel('Number of heads n')
plt.ylabel('Probability')
plt.legend(loc='upper right')
plt.show()

(1) Now, execute the previous cell several times. What is happening? Why?

(2) Increase your sample size and try to judge how many samples would be required so that the obtained statistics follows the expectations. 

(3) Let's go back to the original problem of actin. We wonder how many depolymerization events we can observe during a time window of 10 seconds. The time resolution of your experiment is 100 ms. The depolymerization rate at the (+) end is 3.3 s$^{-1}$. We will assume no depolymerization on the (-) end. By adapting the above code, how many times to you need to repeat the experiment to obtain a reliable Poisson distribution? 

The Poisson distribution function is already present in Python as well. It is accessible using $poisson.pmf(k,\mu)$, which returns the probability of having $k$ events of probability $p$, out of $n$ realizations, with $\mu=p\times n$. to plot the Poisson distribution, you can add the following line when you are doing your figure: 

plt.plot(h_plot, poisson.pmf(h_plot,mu),color='red',marker='o',label='expected',linestyle='')

(4) Compare the Poisson distribution and the binomial distribution expectation. Comment on the possible differences. 

[Bonus question] Estimate the average lifetime of the filament. What is the probability that we still have a polymer after that time if we were to perform an experiment of the exact same length? 

## 2. Stochastic model of gene expression

In this problem, we will consider a very simplistic model of gene expression: 
DNA $\rightarrow$ mRNA $\rightarrow$ proteins. We will generate typical trajectories in the biomolecular phase space, generating the typical number of mRNAs and proteins that would be obtained starting from DNA that is transcribed and translated. 
The time evolution of the system is governed by the following equations: 
\begin{equation}
\frac{dm}{dt} = \beta_m - \gamma_mm\\
\frac{dp}{dt} = \beta_pm - \gamma p\\
\end{equation}
Here, $m$ and $p$ are the concentrations of mRNA and proteins, respectively; $\beta_m$ the rate at which DNA is transcribed; $\beta_p$ the rate at which mRNA is translated; $\gamma_m$ and $\gamma$ the degradation rates of mRNA and proteins. 

For convenience, we will work in reduced units of time, i.e., we set up the timescale to be that of mRNA degradation ($t$ replaces $\gamma_mt$), and we then replace the different parameters by reduced ones: $\beta_m \equiv \beta_m/\gamma_m$, $\beta_p \equiv \beta_p/\gamma_m$, $\gamma \equiv \gamma/\gamma_m$. The dimensionless equations thus become
\begin{equation}
\frac{dm}{dt} = \beta_m - m\\
\frac{dp}{dt} = \beta_pm - \gamma p\\
\end{equation}

We can then write a Master equation for this dynamics, by introducing $P(m,p,t)$ the probability of having $m$ mRNA and $p$ proteins at time $t$. 
\begin{equation}
\frac{dP(m,p,t)}{dt} = \beta_mP(m-1,p,t) + (m+1)P(m+1,p,t) - \beta_mP(m,p,t)-mP(m,p,t) + \beta_pmP(m,p-1,t)+\gamma(p+1)P(m,p+1,t) - \beta_pmP(m,p,t) - \gamma pP(m,p,t)\\
\end{equation}

The Gillespie algorithm (proposed in the 70s), written below, is a way to solve this equation by sampling the trajectory space. You do not need to understand all the details, and you can find some useful documentation about it online if you are interested. 


In [ ]:
simple_update = np.array([[1, 0],   # Make mRNA transcript
                          [-1, 0],  # Degrade mRNA
                          [0, 1],   # Make protein
                          [0, -1]], # Degrade protein
                         dtype=np.int)

In [ ]:
def simple_propensity(propensities, population, t, beta_m, beta_p, gamma):
    """Updates an array of propensities given a set of parameters
    and an array of populations.
    """
    # Unpack population
    m, p = population
    
    # Update propensities
    propensities[0] = beta_m      # Make mRNA transcript
    propensities[1] = m           # Degrade mRNA
    propensities[2] = beta_p * m  # Make protein
    propensities[3] = gamma * p   # Degrade protein

In [ ]:
def sample_discrete(probs):
    """Randomly sample an index with probability given by probs."""
    # Generate random number
    q = np.random.rand()
    
    # Find index
    i = 0
    p_sum = 0.0
    while p_sum < q:
        p_sum += probs[i]
        i += 1
    return i - 1

In [ ]:
def gillespie_draw(propensity_func, propensities, population, t, args=()):
    """
    Draws a reaction and the time it took to do that reaction.
    
    Parameters
    ----------
    propensity_func : function
        Function with call signature propensity_func(population, t, *args)
        used for computing propensities. This function must return
        an array of propensities.
    population : ndarray
        Current population of particles
    t : float
        Value of the current time.
    args : tuple, default ()
        Arguments to be passed to `propensity_func`.
        
    Returns
    -------
    rxn : int
        Index of reaction that occured.
    time : float
        Time it took for the reaction to occur.
    """
    # Compute propensities
    propensity_func(propensities, population, t, *args)
    
    # Sum of propensities
    props_sum = propensities.sum()
    
    # Compute next time
    time = np.random.exponential(1.0 / props_sum)
    
    # Compute discrete probabilities of each reaction
    rxn_probs = propensities / props_sum
    
    # Draw reaction from this distribution
    rxn = sample_discrete(rxn_probs)
    
    return rxn, time

In [ ]:
def gillespie_ssa(propensity_func, update, population_0, time_points, args=()):
    """
    Uses the Gillespie stochastic simulation algorithm to sample
    from probability distribution of particle counts over time.
    
    Parameters
    ----------
    propensity_func : function
        Function of the form f(params, t, population) that takes the current
        population of particle counts and return an array of propensities
        for each reaction.
    update : ndarray, shape (num_reactions, num_chemical_species)
        Entry i, j gives the change in particle counts of species j
        for chemical reaction i.
    population_0 : array_like, shape (num_chemical_species)
        Array of initial populations of all chemical species.
    time_points : array_like, shape (num_time_points,)
        Array of points in time for which to sample the probability
        distribution.
    args : tuple, default ()
        The set of parameters to be passed to propensity_func.        

    Returns
    -------
    sample : ndarray, shape (num_time_points, num_chemical_species)
        Entry i, j is the count of chemical species j at time
        time_points[i].
    """

    # Initialize output
    pop_out = np.empty((len(time_points), update.shape[1]), dtype=np.int)

    # Initialize and perform simulation
    i_time = 1
    i = 0
    t = time_points[0]
    population = population_0.copy()
    pop_out[0,:] = population
    propensities = np.zeros(update.shape[0])
    while i < len(time_points):
        while t < time_points[i_time]:
            # draw the event and time step
            event, dt = gillespie_draw(propensity_func, propensities, population, t, args)
                
            # Update the population
            population_previous = population.copy()
            population += update[event,:]
                
            # Increment time
            t += dt

        # Update the index
        i = np.searchsorted(time_points > t, True)
        
        # Update the population
        pop_out[i_time:min(i,len(time_points))] = population_previous
        
        # Increment index
        i_time = i
                           
    return pop_out

This defines the core of the algorithm. Now, let's see how it runs. 
With $args$, you pass the kinetic parameters  $\beta_m, \beta_p, \gamma$ in reduced unites. $size$ is the number of trajectories that you want to sample, and $time\_points$ will define the total simulation length. 

In [ ]:
# Specify parameters for calculation

betam = 5.
betap=5.
gamma=2.

tmax = 10 
nbin = 101
size = 2


time_points = np.linspace(0, tmax, nbin)
args = (betam, betap, gamma)


population_0 = np.array([0, 0], dtype=int) # initial conditions 


# Seed random number generator for reproducibility
np.random.seed(42)

# Initialize output array
samples = np.empty((size, len(time_points), 2), dtype=int)

# Run the calculations
for i in tqdm.notebook.tqdm(range(size)):
    samples[i,:,:] = gillespie_ssa(simple_propensity, simple_update,
                                population_0, time_points, args=args)
    
# Solve the equations numerically 

# Plot trajectories and mean

for x in samples[:,:,0]:
    plt.plot(time_points, x, linewidth=0.3,alpha=0.8)
plt.plot(time_points, samples[:,:,0].mean(axis=0),linewidth=2,alpha=1,color='red',label='m_avg(t)')
#plt.plot(time_points, m,linewidth=2,alpha=1,color='blue',label='m_ODE(t)')
plt.ylabel('mRNA')
plt.xlabel('time')
plt.legend(loc='best')
plt.show()
    
for x in samples[:,:,1]:
    plt.plot(time_points, x, linewidth=0.3,alpha=0.8)
plt.plot(time_points, samples[:,:,1].mean(axis=0),linewidth=2,alpha=1,color='red',label='p_avg(t)')
#plt.plot(time_points, p,linewidth=2,alpha=1,color='blue',label='m_ODE(t)')
plt.ylabel('proteins')
plt.xlabel('time')
plt.legend(loc='best')
plt.show()
    
    

(1) Play with the number of sampled trajectories. How many are required to obtain some converged and smooth time-dependent populations?

(2) The populations reach a plateau after an initial increase. Was this plateau expected? How is this related to $\beta_m$, $\beta_p$ and $\gamma$?

(3) In fact, the results of these stochastic simulations can be compared with the analytical solutions of the ordinary differential equations (ODE) defining the time evolution of the populations. The following piece of code returns m(t) and p(t) by solving these equations: 

In [ ]:
# function that returns dz/dt
def model(z,t):
    m = z[0]
    p = z[1]
    dmdt = betam-m
    dpdt = betap*m-gamma*p
    dzdt = [dmdt,dpdt]
    return dzdt

# initial condition
z0 = [0,0]
# number of time points
n = 101
# time points
t = np.linspace(0,10,n)
# store solution
m = np.empty_like(t)
p = np.empty_like(t)
# record initial conditions
m[0] = z0[0]
p[0] = z0[1]
# solve ODE
for i in range(1,n):
    # span for next time step
    tspan = [t[i-1],t[i]]
    # solve for next step
    z = odeint(model,z0,tspan)
    # store solution for plotting
    m[i] = z[1][0]
    p[i] = z[1][1]
    # next initial condition
    z0 = z[1]



(4) Re-run the cell propagating the algorithm by integrating this function and compare directly in the plots your results to that of the numerical results of the ODE. 

It is also possible to estimate the standard deviations, which will basically give an idea about the spread in the number of mRNA/proteins you would measure experimentally from a finite number of observations on a limited number of mRNA and protein copies: 

In [ ]:
print('mRNA mean copy number =', samples[:,-50:,0].mean())
print('protein mean copy number =', samples[:,-50:,1].mean())
print('\nmRNA standard deviation =', samples[:,-50:,0].std())
print('protein standard deviation =', samples[:,-50:,1].std())


This approach can thus give some useful information about the probability distributions and about the likelihood of experimental observations, that continuous approaches such as the ODE do not give.

(5) You can play around with the kinetic parameters to realize how they influence en time evolution of the populations, and how they determine e.g. their steady-state values. 

## 3. Rate of barrier crossing

In this problem, we will be running a Langevin dynamics code in order to simulate the diffusion of a model system on a model free-energy surface. Provided with the time-course of the particle position, we will then try to recover the rate at which it crosses the barrier. 

Let's first examine the free-energy surface we will consider. 

In [ ]:
#The double-well potential we will use.

def double_well(x,Uo,a,C):
    k=100000
    xlim=3.9
    V=0
    dV=0
    if x>xlim:
        V=k*(x-xlim)**2
        dV=2*k*(x-xlim)
    if x<-xlim:
        V=k*(x+xlim)**2
        dV=2*k*(x+xlim)
    potential_energy=(Uo/a**4)*(C*x**2-a**2)**2 + V
    return potential_energy

Uo=15000.
a=4
C=2.
L=8.
xs=np.arange(-L/2,L/2,0.001)
U=np.array([double_well(x,Uo,a,C) for x in xs])
print("Barrier Height = ",U[4000] - np.min(U),  " J/mol")
plt.figure()
plt.plot(xs,U)
plt.xlabel("x (Å)")
plt.ylabel("U (J/mol)")
plt.show()

The following cell contains the algorithm itself and its subroutines.


In [ ]:
#Algorithm BAOAB adapted from B Leimkuhler, C Matthews JCP 2013

#Mass in kg/mol, energy in J/mol, T in K, length in Angstrom, time in ps.
def position_update(x,v,dt):
    x_new = x + v*dt/2.
    return x_new

def velocity_update(v,F,M,dt):
    v_new = v + (F/M)*dt/2.
    return v_new

def random_velocity_update(v,gamma,T,M,R,dt):
    xi = np.random.normal()
    c1 = np.exp(-gamma*dt)
    c2 = np.sqrt(1-c1*c1)*np.sqrt(R*T/M)
    v_new = c1*v + xi*c2
    return v_new

def potential(x,Uo,a,C):
    k=100000
    xlim=3.9
    V=0
    dV=0
    if x>xlim:
        V=k*(x-xlim)**2
        dV=2*k*(x-xlim)
    if x<-xlim:
        V=k*(x+xlim)**2
        dV=2*k*(x+xlim)
    potential_energy=(Uo/a**4)*(C*x**2-a**2)**2 + V
    forc = -((4*Uo*C*x)/(a**4))*(C*x**2-a**2) + dV
    return potential_energy,forc


def position_pbc(x,L):
    x_new=x
    if (x>L/2.):
        x_new=x-L
    elif (x<-1*L/2.):
        x_new=x+L
    return x_new

def baoab(L,max_time,dt,R,M,T,d,gamma,Uo,a,C,initial_position,initial_velocity,save_frequency=10, **kwargs ):
    x = initial_position
    v = initial_velocity
    force=0.
    t = 0
    step_number = 0
    positions = []
    velocities = []
    total_energies = []
    save_times = []

    while(t<max_time):

        # B
        potential_energy, force = potential(x,Uo,a,C)
        v = velocity_update(v,force,M,dt)

        #A
        x = position_update(x,v,dt)

        #check PBC (wrap)
        x=position_pbc(x,L)

        #O
        v = random_velocity_update(v,gamma,T,M,R,dt)
        #A
        x = position_update(x,v,dt)

        #check PBC (wrap)
        x=position_pbc(x,L)

        # B
        potential_energy, force = potential(x,Uo,a,C)
        v = velocity_update(v,force,M,dt)

        if step_number%save_frequency == 0 and step_number>0:
            e_total = .5*v*v + potential_energy

            positions.append(x)
            velocities.append(v)
            total_energies.append(e_total)
            save_times.append(t)

        t = t+dt
        step_number = step_number + 1

    return save_times, positions, velocities, total_energies

Now, let's run a Langevin dynamics trajectory per se. The different parameters are first declared, and the self-comments should be pretty explicit. 

In [ ]:
my_L = 8. #angstrom
my_max_time=3000#ps
my_dt = 0.002 #ps
my_T = 300 #K
my_M = 1 #kg/mol
Na= 6.02214076*10**23 #mol^-1
kB = 1.380649*10**(-23) #J/K
h=6.63*10**(-34) #J.s
my_m = my_M/Na #kg
my_D = 0.2 #angstrom²/ps
R = 8.314 #J.mol^-1.K^-1
my_gamma = R*my_T/(my_D*my_M) #ps^-1 
initial_position = 0.
initial_velocity = 0.0
save_times,positions,velocities,total_energies =  baoab(my_L,my_max_time,my_dt,R,my_M,my_T,my_D,my_gamma,Uo,a,C,initial_position,initial_velocity,save_frequency=10)

(1) Given that the particle position is saved every 10 simulation timesteps, plot the position of the particle as a function of time during the first 200 ps of the simulation. The following cell contains an example that you need to complete: 

In [ ]:
xmx= 10000
print("Portion of the trajectory : we can see jumps between the two wells.")
plt.figure()
plt.plot(save_times[0:xmx],positions[0:xmx])
plt.xlabel("Time (ps)")
plt.ylabel("x (Å)")
plt.show()

(2) What kind of distribution do you expect for the particle positions? The following routine allows to perform such calculations; comment and proceed to the necessary changes to your simulation in order to recover results that follow your expectations. 

In [ ]:
prob,bins = np.histogram(positions,bins = 100,density=True)
rplot=0.5*(bins[1:]+bins[:-1])
plt.figure()
plt.plot(rplot,prob)
plt.xlabel("x (Å)")
plt.ylabel("Probability of presence")
plt.show()

Let's now estimate the rate of transition from the left well to the right well. We define state A as any system belonging to the left well. The following routine allows to compute the time correlation for being in A, using absorbing boundary conditions once the barrier to B has been crossed. 

In [ ]:
#Probability to be in A at an instant t:
nt = len(positions)
p_A = []
for t in range(nt):
    if positions[t]<-0.5:
        p_A.append(1)
    else:
        p_A.append(0)
p_A=np.array(p_A)

corr_p = [] #A list of correlation functions (one per instant where the particule is in A)
t_in_A = np.where(p_A==1)[0]  #look for the instant where the particule is in A
for t in t_in_A: #compute the correlation function starting at every instant the particule is in A
    if (t%10==0):
        corr = []
        tau=0
        while ((t+tau<nt-1) & (p_A[t]*p_A[t+tau]==1)): #stop following the particule if goes in B
            corr.append(p_A[t]*p_A[t+tau])
            tau+=1
        corr_p.append(corr)
        
        
max_length = len(corr_p[0])
for c in corr_p:
    if len(c) > max_length:
        max_length=len(c)
        
#Averaging the correlation functions on the starting points

corr_p_avg = np.zeros(max_length)
counts = np.zeros(max_length)
for c in corr_p:
    for j in range(len(c)):
        corr_p_avg[j]+=1
corr_p_avg/=(0.1*len(t_in_A))

time_corr = np.array(save_times[0:max_length])
plt.figure()
plt.plot(time_corr,corr_p_avg)
plt.yscale("log")
plt.xlabel('Time (ps)')
plt.ylabel("C(t)")
plt.show()

(3) Comment on the decay of this correlation function. What does the time constant correspond to? How to compute the rate? 

This is done by the following routine: 

In [ ]:
xmx_fit = 300
xmx_plot = -1
from scipy.optimize import curve_fit
def affine(x,a,b):
    return a*x+b
popt,pcov = curve_fit(affine,time_corr[0:xmx_fit],np.log(corr_p_avg[0:xmx_fit])) #we do an affine fit on the log of the correlation
fv = np.exp(affine(time_corr[0:xmx_fit],popt[0],popt[1]))
plt.figure()
plt.plot(time_corr[0:xmx_plot],corr_p_avg[0:xmx_plot],label = "Simulation")
plt.plot(time_corr[0:xmx_fit],fv,label = "Exponential fit",ls="dashed")
plt.xlabel("Time (ps)")
plt.ylabel(r"$\langle p_{A}(0)p_{A}(t)\rangle$")
plt.yscale("log")
plt.legend()
plt.xlabel('Time (ps)')
plt.ylabel("C(t)")
plt.show()

print("k = ", -popt[0], "ps-1")

(4) What is, in that case, the average lifetime in state A ?

(5) Does the result follow the expectations from the transition state theory? 

(6) By systematically varying the free-energy barrier, investigate whether the rate constant is exponentially dependent on this barrier. 